### About
This notebook contains simple, toy examples to help you get started with FairMLHealth tool usage. This same content is mirrored in the repository's main [README](../README.md).

### Example Setup

In [1]:
from fairmlhealth import report as fr, analyze

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier


np.random.seed(547)
X = pd.DataFrame({'col1': np.random.randint(1, 50, 16), 
                  'col2': np.random.randint(1, 50, 16),
                  'col3': np.random.randint(1, 50, 16),
                  'gender': [0, 1]*8, 
                  'ethnicity': [1, 1, 0, 0]*4,
                  'other': [1, 0, 0, 1]*4
                 })
y = pd.Series([1, 1, 1, 0, 1, 1, 0, 0]*2, name='y')
X_train, y_train= X.iloc[0:5], y.iloc[0:5]
X_test, y_test = X.iloc[5:16], y.iloc[5:16]

#Train models
model_1 = BernoulliNB().fit(X_train, y_train)
model_2 = DecisionTreeClassifier().fit(X_train, y_train)

# Deterimine your set of protected attributes
prtc_attr = X_test['gender']

# Specify either a dict or a list of trained models to compare
model_dict = {'model_1': model_1, 'model_2': model_2}


In [2]:
display(X)

,col1,col2,col3,gender,ethnicity,other
0,48,36,20,0,1,1
1,24,48,44,1,1,0
2,19,18,44,0,0,0
3,38,29,49,1,0,1
4,45,33,23,0,1,1
5,35,43,5,1,1,0
6,24,33,12,0,0,0
7,39,39,46,1,0,1
8,6,11,31,0,1,1
9,36,21,16,1,1,0


### Model Measurement
The primary feature of this library is the model comparison tool. The current version supports assessment of binary prediction models through use of the measure_models and compare_models functions.

Measure_model is designed to generate a report of multiple fairness metrics for a single model. Here it is shown wrapped in a "flag" function to emphasize values that are outside of the "fair" range. 

In [3]:
# Generate a pandas dataframe of measures
fr.measure_model(X_test, y_test, prtc_attr, model_1, flag_oor=True)


### Evaluating

FairMLHealth now also includes stratified table features to aid in identifying the source of unfairness or other bias: data tables, performance tables, and bias tables. Note that these stratified tables can evaluate multiple features at once, and that there are two options for identifying which features to assess.

Note that the flag tool has not yet been updated to work with stratified tables.

#### Stratified Data Tables

The data table is shown below with each of the two data argument options. It evaluates basic statistics specific to each feature-value, in addition to relative statistics for the target value. Since the table can be used to evaluate many features at once, it can be a useful option for identifying patterns of bias either alone or in concert with other (e.g., visual methods).

In [4]:
# Arguments Option 1: pass full set of data, subsetting with *features* argument
analyze.data(X_test, y_test, features=['gender', 'col1'])

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,y Mean,y Median,y Std. Dev.
0,ALL FEATURES,ALL VALUES,11,NaN,0,1.0000,0.5455,1.0,0.5222
1,gender,0,5,0.9940,0,0.4545,0.6000,1.0,0.5477
2,gender,1,6,0.9940,0,0.5455,0.5000,0.5,0.5477
3,col1,2,1,3.2776,0,0.0909,0.0000,0.0,NaN
4,col1,6,1,3.2776,0,0.0909,1.0000,1.0,NaN
5,col1,20,1,3.2776,0,0.0909,1.0000,1.0,NaN
6,col1,24,1,3.2776,0,0.0909,0.0000,0.0,NaN
7,col1,35,1,3.2776,0,0.0909,1.0000,1.0,NaN
8,col1,36,1,3.2776,0,0.0909,1.0000,1.0,NaN
9,col1,37,2,3.2776,0,0.1818,0.5000,0.5,0.7071


In [5]:
# Multiple targets can also be passed through the data table 
analyze.data(X_test, X_test, features=['gender', 'col1'], targets=['col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,col2 Mean,col2 Median,col2 Std. Dev.,col3 Mean,col3 Median,col3 Std. Dev.
0,ALL FEATURES,ALL VALUES,11,NaN,0,1.0000,29.8182,33.0,13.0753,26.3636,22.0,14.7802
1,gender,0,5,0.9940,0,0.4545,27.4000,33.0,15.8524,27.8000,31.0,12.7554
2,gender,1,6,0.9940,0,0.5455,31.8333,34.5,11.4091,25.1667,19.0,17.4059
3,col1,2,1,3.2776,0,0.0909,30.0000,30.0,NaN,47.0000,47.0,NaN
4,col1,6,1,3.2776,0,0.0909,11.0000,11.0,NaN,31.0000,31.0,NaN
5,col1,20,1,3.2776,0,0.0909,16.0000,16.0,NaN,15.0000,15.0,NaN
6,col1,24,1,3.2776,0,0.0909,33.0000,33.0,NaN,12.0000,12.0,NaN
7,col1,35,1,3.2776,0,0.0909,43.0000,43.0,NaN,5.0000,5.0,NaN
8,col1,36,1,3.2776,0,0.0909,21.0000,21.0,NaN,16.0000,16.0,NaN
9,col1,37,2,3.2776,0,0.1818,41.5000,41.5,2.1213,32.0000,32.0,18.3848


In [6]:
# The "ALL FEATURES" overview column can be turned off via the add_overview argument
analyze.data(X_test[['gender']], y_test, add_overview=False)

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,y Mean,y Median,y Std. Dev.
0,gender,0,5,0.994,0,0.4545,0.6,1.0,0.5477
1,gender,1,6,0.994,0,0.5455,0.5,0.5,0.5477


In [7]:
# Multiple targets can also be passed through the data table 
analyze.data(X_test, X_test, features=['gender', 'col1'], targets=['col1', 'col2', 'col3'])

,Feature Name,Feature Value,Obs.,Entropy,Missing Values,Value Prevalence,col1 Mean,col1 Median,col1 Std. Dev.,col2 Mean,col2 Median,col2 Std. Dev.,col3 Mean,col3 Median,col3 Std. Dev.
0,ALL FEATURES,ALL VALUES,11,NaN,0,1.0000,28.9091,36.0,14.0816,29.8182,33.0,13.0753,26.3636,22.0,14.7802
1,col1,2,1,3.2776,0,0.0909,NaN,NaN,NaN,30.0000,30.0,NaN,47.0000,47.0,NaN
2,col1,20,1,3.2776,0,0.0909,NaN,NaN,NaN,16.0000,16.0,NaN,15.0000,15.0,NaN
3,col1,24,1,3.2776,0,0.0909,NaN,NaN,NaN,33.0000,33.0,NaN,12.0000,12.0,NaN
4,col1,35,1,3.2776,0,0.0909,NaN,NaN,NaN,43.0000,43.0,NaN,5.0000,5.0,NaN
5,col1,36,1,3.2776,0,0.0909,NaN,NaN,NaN,21.0000,21.0,NaN,16.0000,16.0,NaN
6,col1,37,2,3.2776,0,0.1818,NaN,NaN,NaN,41.5000,41.5,2.1213,32.0000,32.0,18.3848
7,col1,38,1,3.2776,0,0.0909,NaN,NaN,NaN,42.0000,42.0,NaN,22.0000,22.0,NaN
8,col1,39,1,3.2776,0,0.0909,NaN,NaN,NaN,39.0000,39.0,NaN,46.0000,46.0,NaN
9,col1,44,1,3.2776,0,0.0909,NaN,NaN,NaN,10.0000,10.0,NaN,32.0000,32.0,NaN


#### Stratified Performance Tables

The stratified performance table contains model performance measures specific to each feature-value subset. If prediction probabilities are available to the model, additional ROC AUC and PR AUC values will be included.

In [8]:
analyze.performance(X_test[['gender']], y_test, model_1.predict(X_test), add_overview=False)

,Feature Name,Feature Value,Obs.,Target Mean,Pred. Mean,Accuracy,F1-Score,FPR,Precision,TPR
0,gender,0,5.0,0.6,1.0,0.6,0.7500,1.0,0.6,1.0
1,gender,1,6.0,0.5,1.0,0.5,0.6667,1.0,0.5,1.0


In [9]:
analyze.performance(X_test[['gender']], y_test, 
                           model_1.predict(X_test), y_prob=model_1.predict_proba(X_test)[:,1])

,Feature Name,Feature Value,Obs.,Target Mean,Pred. Mean,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,11.0,0.5455,1.0,0.5455,0.7059,1.0,0.1667,0.5455,0.8667,1.0
1,gender,0,5.0,0.6000,1.0,0.6000,0.7500,1.0,0.3333,0.6000,0.8333,1.0
2,gender,1,6.0,0.5000,1.0,0.5000,0.6667,1.0,0.0000,0.5000,1.0000,1.0


#### Stratified Bias Fairness Tables

The stratified bias table contains model bias measures specific to each feature-value subset. Inspired by common measures of fairness, the tool assumes each feature-value as the "privileged" group relative to all other possible values for the feature. For example, row 3 in the table below displaying measures of "col1" value of "2" where 2 is considered to be the privileged group and all other values (1, 2, 45, and 50) are considered unprivileged.

To simplify the table, fairness measures have been reduced to their component parts. For example, measures of Equalized Odds can be determined by combining the True Positive Rate (TPR) Ratios & Differences with False Positive Rate (FPR) Ratios & Differences.

See also: [Fairness Quick References](../docs/Fairness_Quick_References.pdf) and the [Tutorial for Evaluating Fairness in Binary Classification](./Tutorial-EvaluatingFairnessInBinaryClassification.ipynb)

In [10]:
analyze.bias(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), flag_oor=True)

~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['col3']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,gender,0,0.0000,1.0000,-0.1000,0.8333,0.0000,1.0000,0.0000,1.0000
1,gender,1,0.0000,1.0000,0.1000,1.2000,0.0000,1.0000,0.0000,1.0000
2,col3,12,0.0000,1.0000,0.6000,0.0000,0.0000,1.0000,1.0000,0.0000
3,col3,15,1.0000,0.0000,-0.5000,0.5000,0.0000,1.0000,0.0000,1.0000
4,col3,16,1.0000,0.0000,-0.5000,0.5000,0.0000,1.0000,0.0000,1.0000
5,col3,19,1.0000,0.0000,-0.5000,0.5000,0.0000,1.0000,0.0000,1.0000
6,col3,22,0.0000,1.0000,0.6000,0.0000,0.0000,1.0000,1.0000,0.0000
7,col3,31,1.0000,0.0000,-0.5000,0.5000,0.0000,1.0000,0.0000,1.0000
8,col3,32,1.0000,0.0000,-0.5000,0.5000,0.0000,1.0000,0.0000,1.0000
9,col3,45,0.0000,1.0000,0.6000,0.0000,0.0000,1.0000,1.0000,0.0000


In [11]:
analyze.bias(X_test[['gender', 'col3']], y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test['other'])

~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['col3']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'true_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['gender']. The ratio is ill-defined and being set to 0.0 because 'precision_score' for privileged samples is 0.

  warn(f"Poss

,other,Feature Name,Feature Value,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,gender,0,-1.0000,0.0000,0.6667,3.0000,0.0000,1.0000,0.0000,1.0000
1,0,gender,1,1.0000,0.0000,-0.6667,0.3333,0.0000,1.0000,0.0000,1.0000
2,0,col3,12,0.0000,1.0000,0.8000,0.0000,0.0000,1.0000,1.0000,0.0000
3,0,col3,15,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
4,0,col3,16,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
5,0,col3,19,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
6,0,col3,45,0.0000,1.0000,0.8000,0.0000,0.0000,1.0000,1.0000,0.0000
7,0,col3,5,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
8,1,gender,0,1.0000,0.0000,-1.0000,0.0000,0.0000,1.0000,-1.0000,0.0000
9,1,gender,1,-1.0000,0.0000,1.0000,0.0000,0.0000,1.0000,1.0000,0.0000


In [12]:
analyze.bias(X_test['col3'].rename("ethnicityCode"), y_test, model_1.predict(X_test), 
                    flag_oor=True, cohorts=X_test['other'].rename("isGenderMale"))

~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['ethnicityCode']. The ratio is ill-defined and being set to 0.0 because 'false_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['ethnicityCode']. The ratio is ill-defined and being set to 0.0 because 'precision_score' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")
~/repos/fairMLHealth/fairmlhealth/__utils.py:48: UserWarning: Possible error in column(s) ['ethnicityCode']. The ratio is ill-defined and being set to 0.0 because 'true_positive_rate' for privileged samples is 0.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,isGenderMale,Feature Name,Feature Value,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,0,ethnicityCode,12,0.0000,1.0000,0.8000,0.0000,0.0000,1.0000,1.0000,0.0000
1,0,ethnicityCode,15,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
2,0,ethnicityCode,16,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
3,0,ethnicityCode,19,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
4,0,ethnicityCode,45,0.0000,1.0000,0.8000,0.0000,0.0000,1.0000,1.0000,0.0000
5,0,ethnicityCode,5,1.0000,0.0000,-0.4000,0.6000,0.0000,1.0000,0.0000,1.0000
6,1,ethnicityCode,22,0.0000,1.0000,0.5000,0.0000,0.0000,1.0000,1.0000,0.0000
7,1,ethnicityCode,31,1.0000,0.0000,-0.7500,0.2500,0.0000,1.0000,0.0000,1.0000
8,1,ethnicityCode,32,1.0000,0.0000,-0.7500,0.2500,0.0000,1.0000,0.0000,1.0000
9,1,ethnicityCode,46,0.0000,1.0000,0.5000,0.0000,0.0000,1.0000,1.0000,0.0000


### Comparing Results for Multiple Models

The compare_models feature can be used to generate side-by-side fairness comparisons of multiple models. Model performance metrics such as accuracy and precision are also provided to facilitate comparison.   

Below is an example output comparing the two example models defined above. Missing values have been added for metrics requiring prediction probabilities, which the second model does not have (note the warning below).

In [13]:
# Pass the data and models to the compare models function, as above
fr.compare_models(X_test, y_test, prtc_attr, model_dict, flag_oor=True)

The compare_models function can also be used to measure two different protected attributes. Protected attributes are measured separately and cannot yet be combined together with this tool.

In [14]:
fr.compare_models(X_test, y_test, 
                    [X_test['gender'], X_test['ethnicity']], 
                    {'gender':model_1, 'ethnicity':model_1},
                    flag_oor=True)
